# Importing libraries and packages

In [1]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder 

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer

import pickle

# Creating Dataset

- Dataset was first convert to csv file with 2 columns Tags and Word
- File was imported, Cleaned and Saved

In [2]:
dataset = pd.read_csv("dataset.csv")
dataset = dataset.dropna()
dataset.to_csv("dataset.csv", index = False)

dataset = pd.read_csv("dataset_test.csv")

dataset = dataset.dropna()
dataset.to_csv("dataset_test.csv", index = False)

# Creating data and labels

- Dataset was divided 4 file
1. X_train : Contains Training Words
2. X_test : Contains Training Tags
3. Y_train : Contains Test Words
4. Y_test : Contains Test Tags

In [3]:
print('Loading data...')

x_train = pd.read_csv("x_train.csv")
x_train["Word"] = x_train["Word"].apply(str)

x_test = pd.read_csv("x_test.csv")
x_test["Word"] = x_test["Word"].apply(str)

y_train = pd.read_csv("y_train.csv")
y_test = pd.read_csv("y_test.csv")

Loading data...


# Label Encoding

- Tags are labelled using label encoder and one hot encoder 

In [4]:
labelencoder = LabelEncoder()

y_train["Tag"] = labelencoder.fit_transform(y_train["Tag"])
y_test["Tag"] = labelencoder.transform(y_test["Tag"])

onehotencoder = OneHotEncoder() 

Y_train = onehotencoder.fit_transform(y_train["Tag"].values.reshape(-1,1)).toarray()
Y_test = onehotencoder.transform(y_test["Tag"].values.reshape(-1,1)).toarray()

# Dataset Preprocessing

- Tokenisation of words and padding

In [5]:
max_features = 80000
maxlen = 1
batch_size = 32

x_train = x_train['Word'].fillna('').tolist()

x_test = x_test['Word'].fillna('').tolist()

from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = 200000)
tokenizer.fit_on_texts(x_train)

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

70525 train sequences
14256 test sequences
Pad sequences (samples x time)
x_train shape: (70525, 1)
x_test shape: (14256, 1)


# Building Model

- Building LSTM model using Tensorflow and Keras

In [6]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(17, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build model...


# Training and Predicting

In [7]:
print('Train...')
model.fit(x_train, Y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, Y_test))

Train...
Epoch 1/3
2204/2204 [==============================] - 215s 96ms/step - loss: 1.2458 - accuracy: 0.6703 - val_loss: 0.6896 - val_accuracy: 0.8043
Epoch 2/3
2204/2204 [==============================] - 224s 102ms/step - loss: 0.6040 - accuracy: 0.8296 - val_loss: 0.6392 - val_accuracy: 0.8095
Epoch 3/3
2204/2204 [==============================] - 216s 98ms/step - loss: 0.5431 - accuracy: 0.8413 - val_loss: 0.6230 - val_accuracy: 0.8166


In [11]:
pred = model.predict(x_test)

# Evaluation of model

In [12]:
score, acc = model.evaluate(pred, Y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

446/446 [==============================] - 3s 6ms/step - loss: 2.8266 - accuracy: 0.6074
Test score: 2.8265540599823
Test accuracy: 0.6073933839797974


# Saving model and other objects

In [14]:
model.save("rnn.hdf5")

with open('labelencoder', 'wb') as f:
    pickle.dump(labelencoder, f)
    
with open('onehotencoder', 'wb') as f:
    pickle.dump(onehotencoder, f)
    
with open('tokenizer', 'wb') as f:
    pickle.dump(tokenizer, f)
